# mlrose Generator and Runner Usage Examples - Andrew Rollings

## Overview

These examples will not solve assignment 2 for you, but they will give you
some idea on how to use the problem generator and runner classes.

Hopefully this will result in slightly fewer
"How do I &lt;insert basic usage here&gt;?" questions every semester...

Also, and in case it hasn't been made clear enough by the TAs, using any of the visualization code in here for your
assignment is a bad idea, for two reasons... (1) It provides nothing useful as far as the assignment goes,
and (2) the TAs will undoubtedly frown upon it.

Visualization is part of the analysis and, for the most part, you're supposed to do that by yourself. Just including
images of the before/after state of a problem really isn't useful in terms of what you're supposed to be analyzing.

### Import Libraries

In [1]:
from IPython.core.display import display, HTML # for some notebook formatting.

import mlrose
import numpy as np
import logging
import networkx as nx
import matplotlib.pyplot as plt
import string


from ast import literal_eval
import chess

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from mlrose import QueensGenerator, MaxKColorGenerator, TSPGenerator
from mlrose import SARunner, GARunner, NNGSRunner

# switch off the chatter
logging.basicConfig(level=logging.WARNING)

/var/folders/q2/n6gs30t12tddx_g65ffkf_h4w_msz4/T/ipykernel_60419/596531505.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML # for some notebook formatting.


ModuleNotFoundError: No module named 'chess'

### Example 1: Generating and Running 8-Queens using the SA algorithm

In [ ]:
# Generate a new 8-Queen problem using a fixed seed.
problem = QueensGenerator().generate(seed=123456, size=8)

The initial state is as follows:

In [ ]:
state = problem.get_state()
print(state)
board_layout = '/'.join([''.join(([str(s)] if s > 0 else []) + ['Q'] + ([str((7-s))] if s < 7 else [])) for s in state])
board = chess.Board(board_layout)
board

In [ ]:
# create a runner class and solve the problem
sa = SARunner(problem=problem,
              experiment_name='queens8_sa',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=123456,
              iteration_list=2 ** np.arange(11),
              max_attempts=500,
              temperature_list=[0.1, 0.5, 0.75, 1.0, 2.0, 5.0],
              decay_list=[mlrose.GeomDecay])

# the two data frames will contain the results
df_run_stats, df_run_curves = sa.run()

The preceding code will run the `SA` algorithm six times for at most 1024 iterations.
Each run is initialized with the temperature specified in the `temperature_list`
using the temperature values specified.

If the fitness remains static for `max_attempts` iterations, it will terminate that run.

Note that the initial state parameters here are just toy values picked specifically
for this example. You will have to choose your own range of values for your
assignment. I strongly recommend you don't just copy these, or you will find
that the grading is unlikely to go the way you would like.

The output in the `df_run_stats` dataframe contains snapshots of the state of the algorithm at the iterations
specified in the `iteration_list` passed into the runner class.

The first 12 rows (corresponding to the first run of this algorithm) are as follows:

In [ ]:
HTML(df_run_stats[['Iteration', 'Fitness', 'FEvals', 'Time', 'State']][0:12].to_html())

The state information is excluded from the previous output.

A sample of this (based on the state of the `GeomDecay` object) is below:

In [ ]:
state_sample = df_run_stats[['schedule_current_value', 'schedule_init_temp', 'schedule_min_temp']][:1]
HTML(state_sample.to_html())

So, to pick out the most performant run from the dataframe, you need to find the row with the best fitness.
As 8-Queens is a minimization problem, you'd pick the row with the minimum fitness.

However, I'm going to look in the `run_curves` (which stores minimal basic information every iteration) to
find out which input state achieved the best fitness in the fewest fitness evaluations.

In [ ]:
best_fitness = df_run_curves['Fitness'].min()
best_runs = df_run_curves[df_run_curves['Fitness'] == best_fitness]

HTML(best_runs.to_html())

This gives us five candidates for the best run. We are going to pick the one with
that reached the best fitness value in the fewest number of evaluations.

(We could also have chosen to use `Iterations` as our criteria.)

In [ ]:
minimum_evaluations = best_runs['FEvals'].min()

best_curve_run = best_runs[best_runs['FEvals'] == minimum_evaluations]

The best run using these criteria is as follows:

In [ ]:
HTML(best_curve_run.to_html())

Which has the following identifying state information:

In [ ]:
best_init_temperature = best_curve_run['Temperature'].iloc()[0].init_temp

print(f'Best initial temperature: {best_init_temperature}')


To map this back to the `run_stats` we look at the configuration data included in
the curve data. The curve data includes at least the minimum identifying information
to determine which run each row came from.

In this case, the value we are looking for is the `Temperature`, which is the initial temperature
used to initialize the `GeomDecay` object.

So, in this case, we are looking for all rows in `df_run_stats` where the temperature is equal to 1.

In [ ]:
run_stats_best_run = df_run_stats[df_run_stats['schedule_init_temp'] == best_init_temperature]
HTML(run_stats_best_run[['Iteration', 'Fitness', 'FEvals', 'Time', 'State']].to_html())

And the best state associated with this is:

In [ ]:
best_state = run_stats_best_run[['schedule_current_value', 'schedule_init_temp', 'schedule_min_temp']].tail(1)
HTML(best_state.to_html())

The final state is as follows:

In [ ]:
state = literal_eval(run_stats_best_run['State'].tail(1).values[0])
print(state)
board_layout = '/'.join([''.join(([str(s)] if s > 0 else []) + ['Q'] + ([str((7-s))] if s < 7 else [])) for s in state])
board = chess.Board(board_layout)
board

### Example 2: Generating and Running Max K Color using the GA algorithm

In [ ]:
# Generate a new Max K problem using a fixed seed.
problem = MaxKColorGenerator().generate(seed=123456, number_of_nodes=10, max_connections_per_node=3, max_colors=3)

The input graph generated for the problem looks like this:

In [ ]:
nx.draw(problem.source_graph,
        pos=nx.spring_layout(problem.source_graph, seed = 3))
plt.show()

In [ ]:
# create a runner class and solve the problem
ga = GARunner(problem=problem,
              experiment_name='max_k_ga',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=123456,
              iteration_list=2 ** np.arange(11),
              population_sizes=[10, 20, 50],
              mutation_rates=[0.1, 0.2, 0.5])

# the two data frames will contain the results
df_run_stats, df_run_curves = ga.run()

The preceding code will run the `GA` algorithm nine times for at most 1024 iterations per run.
Each run is a permutation from the list of `population_sizes` and `mutation_rates`.

Note that the initial state parameters here are just toy values picked specifically
for this example. You will have to choose your own range of values for your
assignment. I strongly recommend you don't just copy these, or you will find
that the grading is unlikely to go the way you would like.

Really. I mean it... A mutation rate of 0.5 is little better than a pure random search.

The output in the `df_run_stats` dataframe contains snapshots of the state of the algorithm at the iterations
specified in the `iteration_list` passed into the runner class.

The first row (corresponding to the first run of this algorithm) are as follows:

In [ ]:
HTML(df_run_stats[['Iteration', 'Fitness', 'FEvals', 'Time', 'State']][0:1].to_html())

The state information is excluded from the previous output.

A sample of this is below:

In [ ]:
state_sample = df_run_stats[['Population Size', 'Mutation Rate']][:1]
HTML(state_sample.to_html())


So, to pick out the most performant run from the dataframe, you need to find the row with the best fitness.
As Max-K-Color is a minimization problem, you'd pick the row with the minimum fitness.

However, I'm going to look in the `run_curves` (which stores minimal basic information every iteration) to
find out which input state achieved the best fitness in the fewest fitness evaluations.

In [ ]:
best_fitness = df_run_curves['Fitness'].min()
best_runs = df_run_curves[df_run_curves['Fitness'] == best_fitness]

HTML(best_runs.to_html())

This gives us nine candidates for the best run. We are going to pick the one with
that reached the best fitness value in the fewest number of evaluations.

(We could also have chosen to use `Iterations` as our criteria.)

In [ ]:
minimum_evaluations = best_runs['FEvals'].min()

best_curve_run = best_runs[best_runs['FEvals'] == minimum_evaluations]

The best runs using these criteria is as follows:

In [ ]:
HTML(best_curve_run.to_html())

We will arbitrarily pick the first row for this example,
which has the following identifying state information:

In [ ]:
best_mr = best_curve_run['Mutation Rate'].iloc()[0]
best_pop_size = best_curve_run['Population Size'].iloc()[0]
print(f'Best Mutation Rate: {best_mr}, best Population Size: {best_pop_size}')


To map this back to the `run_stats` we look at the configuration data included in
the curve data. The curve data includes at least the minimum identifying information
to determine which run each row came from.

In this case, the values we are looking for are the `Mutation Rate` and `Population Size`.

So, we are looking for all rows in `df_run_stats` where the mutation rate and population size are equal to our best values.

In [ ]:
run_stats_best_run = df_run_stats[(df_run_stats['Mutation Rate'] == best_mr) & (df_run_stats['Population Size'] == best_pop_size)]
HTML(run_stats_best_run[['Iteration', 'Fitness', 'FEvals', 'Time']].to_html())

And the best state associated with this is:

In [ ]:
best_state = run_stats_best_run[['State']].tail(1)
HTML(best_state.to_html())

For the following node ordering:

In [ ]:
print([n for n in problem.source_graph.nodes])

Reordering the state by ascending node number gives the following:

In [ ]:
color_indexes = literal_eval(run_stats_best_run['State'].tail(1).values[0])
ordered_state = [color_indexes[n] for n in problem.source_graph.nodes]
print(ordered_state)

Which results in a graph looking like this:

In [ ]:
colors = ['lightcoral', 'lightgreen', 'yellow']
node_color_map = [colors[s] for s in ordered_state]

nx.draw(problem.source_graph,
        pos=nx.spring_layout(problem.source_graph, seed = 3),
        with_labels=True,
        node_color=node_color_map)
plt.show()

### Example 3: Generating and Running TSP using the GA algorithm

In [ ]:
# Generate a new TSP problem using a fixed seed.
problem = TSPGenerator().generate(seed=123456, number_of_cities=20)

The input graph generated for the problem looks like this:

In [ ]:
fig, ax = plt.subplots(1)         # Prepare 2 plots
ax.set_yticklabels([])
ax.set_xticklabels([])
for i, (x,y) in enumerate(problem.coords):
    ax.scatter(x,y, s=200, c='cornflowerblue')             # plot A
node_labels = {k:str(v) for k, v in enumerate(string.ascii_uppercase) if k < len(problem.source_graph.nodes)}
for i in node_labels.keys():
    x,y = problem.coords[i]
    plt.text(x, y, node_labels[i], ha="center", va="center", c='white', fontweight='bold',
             bbox = dict(boxstyle=f"circle,pad=0.15", fc='cornflowerblue'))

plt.tight_layout()
plt.show()

In [ ]:
# create a runner class and solve the problem
ga = GARunner(problem=problem,
              experiment_name='tsp_ga',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=123456,
              iteration_list=2 ** np.arange(11),
              population_sizes=[10, 20],
              mutation_rates=[0.1, 0.25, 0.5])

# the two data frames will contain the results
df_run_stats, df_run_curves = ga.run()

The preceding code will run the `GA` algorithm nine times for at most 1024 iterations per run.
Each run is a permutation from the list of `population_sizes` and `mutation_rates`.

Note that the initial state parameters here are just toy values picked specifically
for this example. You will have to choose your own range of values for your
assignment. I strongly recommend you don't just copy these, or you will find
that the grading is unlikely to go the way you would like.

Really. I mean it... A mutation rate of 0.5 is little better than a pure random search.

The output in the `df_run_stats` dataframe contains snapshots of the state of the algorithm at the iterations
specified in the `iteration_list` passed into the runner class.

The first row (corresponding to the first run of this algorithm) are as follows:

In [ ]:
HTML(df_run_stats[['Iteration', 'Fitness', 'FEvals', 'Time', 'State']][0:1].to_html())

The state information is excluded from the previous output.

A sample of this is below:

In [ ]:
state_sample = df_run_stats[['Population Size', 'Mutation Rate']][:1]
HTML(state_sample.to_html())


So, to pick out the most performant run from the dataframe, you need to find the row with the best fitness.
As TSP is a minimization problem, you'd pick the row with the minimum fitness.

However, I'm going to look in the `run_curves` (which stores minimal basic information every iteration) to
find out which input state achieved the best fitness in the fewest fitness evaluations.

In [ ]:
best_fitness = df_run_curves['Fitness'].min()
best_runs = df_run_curves[df_run_curves['Fitness'] == best_fitness]

HTML(best_runs[0:10].to_html())

This gives us nine candidates for the best run. We are going to pick the one with
that reached the best fitness value in the fewest number of evaluations.

(We could also have chosen to use `Iterations` as our criteria.)

In [ ]:
minimum_evaluations = best_runs['FEvals'].min()

best_curve_run = best_runs[best_runs['FEvals'] == minimum_evaluations]

The best runs using these criteria is as follows:

In [ ]:
HTML(best_curve_run.to_html())

This has the following identifying state information:

In [ ]:
best_mr = best_curve_run['Mutation Rate'].iloc()[0]
best_pop_size = best_curve_run['Population Size'].iloc()[0]
print(f'Best Mutation Rate: {best_mr}, best Population Size: {best_pop_size}')


To map this back to the `run_stats` we look at the configuration data included in
the curve data. The curve data includes at least the minimum identifying information
to determine which run each row came from.

In this case, the values we are looking for are the `Mutation Rate` and `Population Size`.

So, we are looking for all rows in `df_run_stats` where the mutation rate and population size are equal to our best values.

In [ ]:
run_stats_best_run = df_run_stats[(df_run_stats['Mutation Rate'] == best_mr) & (df_run_stats['Population Size'] == best_pop_size)]
HTML(run_stats_best_run[['Iteration', 'Fitness', 'FEvals', 'Time']].to_html())

And the best state associated with this is:

In [ ]:
best_state = run_stats_best_run[['State']].tail(1)
HTML(best_state.to_html())

Which results in a graph looking like this:

In [ ]:
ordered_state = literal_eval(run_stats_best_run['State'].tail(1).values[0])
edge_labels = {(ordered_state[i], ordered_state[(i+1) % len(ordered_state)]):f'{str(i+1)}➜' for i in range(len(ordered_state))}
# print(ordered_state)

In [ ]:
fig, ax = plt.subplots(1)         # Prepare 2 plots
ax.set_yticklabels([])
ax.set_xticklabels([])
for i, (x,y) in enumerate(problem.coords):
    ax.scatter(x,y, s=1,c='green' if i == 5 else 'cornflowerblue')             # plot A


for i in range(len(ordered_state)):
    start_node = ordered_state[i]
    end_node = ordered_state[(i+1) % len(ordered_state)]
    start_pos = problem.coords[start_node]
    end_pos = problem.coords[end_node]
    ax.annotate("",
            xy=start_pos, xycoords='data',
            xytext=end_pos, textcoords='data',
            c='red',
            arrowprops=dict(arrowstyle="->",
                            ec='red',
                            connectionstyle="arc3"))
node_labels = {k:str(v) for k, v in enumerate(string.ascii_uppercase) if k < len(problem.source_graph.nodes)}

for i in node_labels.keys():
    x,y = problem.coords[i]
    plt.text(x, y, node_labels[i], ha="center", va="center", c='white', fontweight='bold',
             bbox = dict(boxstyle=f"circle,pad=0.15", fc='green' if i == ordered_state[0] else 'cornflowerblue'))

plt.tight_layout()
plt.show()

And, to verify that the route is correct (or at least, the shortest one found):

In [ ]:
all_edge_lengths = {(x, y):d for x, y, d in problem.distances}
all_edge_lengths.update({(y, x):d for x, y, d in problem.distances})

route_length = sum([all_edge_lengths[k] for k in edge_labels.keys()])
print(f'route_length: ({round(route_length, 6)}) equal to best_fitness: ({round(best_fitness, 6)})')

### Example 4: Using the NNGSRunner with the RHC algorithm

In [ ]:
# Load and Split data into training and test sets
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size = 0.3, random_state = 123456
)

# Normalize feature data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# One hot encode target values
one_hot = OneHotEncoder()
y_train_hot = one_hot.fit_transform(y_train.reshape(-1, 1)).todense()
y_test_hot = one_hot.transform(y_test.reshape(-1, 1)).todense()

grid_search_parameters = {
    'max_iters': [1000],                          # nn params
    'learning_rate': [1e-2],                       # nn params
    'activation': [mlrose.relu],            # nn params
    'restarts': [1],                             # rhc params
}

nnr = NNGSRunner(
    x_train=X_train_scaled,
    y_train=y_train_hot,
    x_test=X_test_scaled,
    y_test=y_test_hot,
    experiment_name='nn_test_rhc',
    algorithm=mlrose.algorithms.rhc.random_hill_climb,
    grid_search_parameters=grid_search_parameters,
    iteration_list=[1, 10, 50, 100, 250, 500, 1000],
    hidden_layer_sizes=[[2]],
    bias=True,
    early_stopping=True,
    clip_max=5,
    max_attempts=500,
    n_jobs=5,
    seed=123456,
    output_directory=None
)


run_stats_df, curves_df, cv_results_df, grid_search_cv = nnr.run()

The runner returns the `run_stats` and `curves` corresponding to *best* hyperparameter combination,
as well as the cross validation results and the underlying `GridSearchCV` object used in the run.

In [ ]:
y_test_pred = grid_search_cv.predict(X_test_scaled)
y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
print(y_test_accuracy)


In [ ]:
y_train_pred = grid_search_cv.predict(X_train_scaled)
y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
print(y_train_accuracy)

In [ ]:
HTML(run_stats_df[['current_restart', 'Iteration', 'Fitness', 'FEvals', 'Time', 'learning_rate']][0:14].to_html())

In [ ]:
HTML(curves_df[['current_restart', 'Iteration', 'Fitness', 'FEvals', 'Time', 'learning_rate']][0:20].to_html())

In [ ]:
HTML(cv_results_df[['mean_test_score', 'rank_test_score', 'mean_train_score', 'param_activation', 'param_hidden_layer_sizes', 'param_learning_rate',
                    'param_max_iters', 'param_restarts']].to_html())